In [37]:
import psycopg2
import os
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import dotenv
import numpy as np
import random
from datetime import datetime, timedelta, timezone
from tqdm import tqdm

In [2]:
dotenv.load_dotenv('.env', override=True)

# Connect to DB
conn = psycopg2.connect(
    f"dbname='{os.environ['POSTGRES_DB']}' user='{os.environ['POSTGRES_USER']}' "
    f"host='{os.environ['POSTGRES_HOST']}' password='{os.environ['POSTGRES_PASSWORD']}'"
)

In [3]:
from_date = "2024-10-30 06:00:00"
to_date = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

In [38]:
def showTracksInArea(left, right, top, bottom, from_date, to_date, show_names=True, title="",
                     speed_limit=1, width_limit=1, resolution="f", frame_interval_minutes=30,
                     history_hours=2, dir_name="animation", interpolation_frames=0):
    from_date = datetime.strptime(
        from_date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)
    to_date = datetime.strptime(
        to_date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timezone.utc)

    frames_dir = f"frames/{dir_name}"
    os.makedirs(frames_dir, exist_ok=True)

    with conn.cursor() as cursor:
        cursor.execute(f"""
            SELECT ships.name, ships.id, location[0] AS latitude, location[1] AS longitude, positions.parsed_date
            FROM positions
            JOIN ships ON positions.ship_id = ships.id
            WHERE positions.location[0] BETWEEN {bottom} AND {top}
            AND positions.location[1] BETWEEN {left} AND {right}
            AND ships.width > {width_limit}
            AND positions.speed > {speed_limit}
            AND positions.parsed_date > '{from_date}'::timestamp
            AND positions.parsed_date <= '{to_date}'::timestamp
            ORDER BY ships.name, positions.parsed_date
        """)
        data = cursor.fetchall()

    ship_data = {}
    for row in data:
        name, id, lat, lon, parsed_date = row
        if id not in ship_data:
            ship_data[id] = {'name': name, 'locations': []}
        ship_data[id]['locations'].append((lat, lon, parsed_date))

    history_duration = timedelta(hours=history_hours)
    frame_interval = timedelta(minutes=frame_interval_minutes / (interpolation_frames + 1))
    total_frames = int((to_date - from_date) / frame_interval)

    frame_count = 0
    current_time = from_date
    with tqdm(total=total_frames, desc="Rendering frames") as pbar:
        while current_time < to_date:
            history_start = max(current_time - history_duration, from_date)
            interval_end = current_time + frame_interval

            plt.figure(figsize=(12, 12 * (top - bottom) / (right - left)))
            m = Basemap(projection='merc',
                        llcrnrlat=bottom, urcrnrlat=top,
                        llcrnrlon=left, urcrnrlon=right,
                        resolution=resolution)
            m.drawcoastlines()
            m.drawcountries()
            m.drawmapboundary(fill_color='aqua')
            m.fillcontinents(color='lightgreen', lake_color='aqua')

            map_width_km = 0.1 * abs(right - left) * 111
            scale_length = max(round(map_width_km, -1), 5)

            m.drawmapscale(
                right - 0.2 * abs(right - left),
                bottom + 0.1 * abs(top - bottom),
                left, top,
                length=scale_length,
                barstyle='fancy', labelstyle='simple', units='km', fontsize=6
            )

            for id, info in ship_data.items():
                interpolated_locations = []
                for i in range(len(info['locations']) - 1):
                    start = info['locations'][i]
                    end = info['locations'][i + 1]
                    interpolated_locations.append(start)
                    time_delta = (end[2] - start[2]) / (interpolation_frames + 1)
                    for j in range(1, interpolation_frames + 1):
                        lat = start[0] + (end[0] - start[0]) * j / (interpolation_frames + 1)
                        lon = start[1] + (end[1] - start[1]) * j / (interpolation_frames + 1)
                        parsed_date = start[2] + time_delta * j
                        interpolated_locations.append((lat, lon, parsed_date))
                interpolated_locations.append(info['locations'][-1])

                locations = [(lat, lon) for lat, lon, date in interpolated_locations
                             if history_start <= date < interval_end]
                if len(locations) > 1:
                    lats, lons = zip(*locations)
                    x, y = m(lons, lats)
                    random.seed(info["name"])
                    color = (random.random(), random.random(), random.random())
                    m.plot(x, y, linewidth=1, color=color)
                    m.plot(x[-1], y[-1], marker='x',
                           markersize=3, linewidth=1, color=color)

                    if show_names:
                        plt.text(
                            x[-1], y[-1], f'{info["name"]}', fontsize=6, color=color, ha='right')

            plt.text(0.01, 0.98, f'Area: {top}, {left}; {right}, {bottom}\n'
                     f'Current frame time: {current_time.strftime("%Y-%m-%d %H:%M:%S")} UTC+0\n'
                     f'Number of ships: {len([info for info in ship_data.values() if any(history_start <= date < interval_end for _, _, date in info["locations"])])}\n'
                     f'Track points in frame: {sum(1 for info in ship_data.values() for _, _, date in info["locations"] if history_start <= date < interval_end)}',
                     fontsize=8, color='black', ha='left', va="top", transform=plt.gca().transAxes,
                     bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

            plt.title(f"Tracks of Ships in {title}")
            plt.savefig(f"{frames_dir}/frame_{frame_count}.png", bbox_inches='tight')
            plt.close()

            current_time = interval_end
            frame_count += 1
            pbar.update(1)

    print(f"{frame_count} frames saved to {frames_dir}")


In [39]:
showTracksInArea(131.653822, 132.289930, 43.324740, 42.837461,
                 from_date=from_date, to_date=to_date,
                 title="Vladivostok area", show_names=False, resolution="i", frame_interval_minutes=15,
                 history_hours=2, dir_name="vladivostok_area", interpolation_frames=4)

Rendering frames:  78%|███████▊  | 62/80 [01:32<00:25,  1.41s/it]

In [40]:
showTracksInArea(131.811162, 131.963941, 43.118932, 43.047156,
                 from_date="2024-10-30 08:00:00", to_date="2024-10-30 12:00:00",
                 title="Port of Vladivostok", show_names=True, resolution="i", frame_interval_minutes=15,
                 history_hours=2, dir_name="vladivostok_port", interpolation_frames=4)

Rendering frames: 26it [00:36,  1.40s/it]                        

26 frames saved to frames/vladivostok_port


In [41]:
showTracksInArea(139.587235, 140.124683, 35.723855, 35.079824,
                 from_date=from_date, to_date=to_date,
                 title="Tokyo area", show_names=True, resolution="i", frame_interval_minutes=15,
                 history_hours=2, dir_name="tokyo_area", interpolation_frames=4)

Rendering frames: 26it [00:36,  1.39s/it]                        

26 frames saved to frames/vladivostok_port


In [42]:
showTracksInArea(139.620327, 140.142928, 35.688097, 35.430781,
                 from_date=from_date, to_date=to_date,
                 title="Port of Tokyo", show_names=True, resolution="i", frame_interval_minutes=15,
                 history_hours=2, dir_name="tokyo_port", interpolation_frames=4)

Rendering frames: 26it [00:39,  1.51s/it]                        

26 frames saved to frames/tokyo_port


In [ ]:
showTracksInArea(132.051025, 135.503411, 34.882905, 33.231879,
                 from_date="2024-10-29 12:00:00", to_date="2024-10-30 13:12:00",
                 title="Seto Inland Sea", show_names=False, resolution="i", frame_interval_minutes=15, history_hours=2, dir_name="seto_inland_sea", interpolation_frames=4)

Rendering frames:   9%|▊         | 43/504 [01:52<21:07,  2.75s/it]